<a href="https://colab.research.google.com/github/opsifiz/AIB5_Emotion-Classification/blob/main/model/test_model_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown https://drive.google.com/uc?id=1mYE-g-9pjFsCv2Eja7byhcEmEPyc9_JB

Downloading...
From (original): https://drive.google.com/uc?id=1mYE-g-9pjFsCv2Eja7byhcEmEPyc9_JB
From (redirected): https://drive.google.com/uc?id=1mYE-g-9pjFsCv2Eja7byhcEmEPyc9_JB&confirm=t&uuid=5c6a7f0c-868c-4cd9-b939-87c3049a1b03
To: /content/my_model02.zip
100% 247M/247M [00:02<00:00, 112MB/s] 


In [3]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.0 MB/s eta 0:00:00


In [4]:
import zipfile

with zipfile.ZipFile("my_model02.zip", 'r') as zip_ref:
    zip_ref.extractall("my_model")

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import contractions
import html
import re

model_dir = "my_model"  # Path to your unzipped model directory

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, trust_remote_code=True)


In [12]:
id2label = {
    0: 'Anxiety',
    1: 'BPD',
    2: 'Normal',
    3: 'Stress',
    4: 'Suicidal',
    5: 'Bipolar',
    6: 'Depression',
    7: 'Mentalillness',
    8: 'Schizophrenia'
}

In [7]:
# Inference function
def predict_status(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=1).item()
    return id2label[predicted_class]

In [8]:
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = contractions.fix(text)
    text = html.unescape(text)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
# Example usage
text = input("Let's Analysis Your text! : ")
text = clean_text(text)
diagnosis = predict_status(text)
print(f"Predicted diagnosis: {diagnosis}")

Let's Analysis Your text! : Hello World!
Predicted diagnosis: Normal


In [14]:
import json

msgs = {
  "BPD": [
    "I feel so intensely and then suddenly feel nothing at all.",
    "One moment I trust you, the next I’m terrified you’ll leave me.",
    "I hate myself for pushing people away, even when I need them most.",
    "Why do my emotions change so fast? I can’t keep up.",
    "Sometimes I just want someone to tell me they won’t leave.",
    "I’m scared of abandonment even when nothing has happened.",
    "My relationships always feel like a rollercoaster.",
    "It’s like I don’t know who I really am.",
    "Everything is either all good or all bad—there’s no in-between.",
    "I feel broken and I don’t know how to fix it."
  ],
  "Anxiety": [
    "What if something goes wrong? I can’t stop thinking about it.",
    "I feel like my heart is racing all the time.",
    "Even small decisions feel overwhelming.",
    "I constantly replay conversations in my head.",
    "I’m always worried about what others think of me.",
    "It feels like I’m stuck in a loop of fear.",
    "I can’t sleep because my mind won’t shut off.",
    "I keep imagining worst-case scenarios.",
    "I feel like something bad is about to happen—even when everything’s fine.",
    "I wish I could relax, but I just can’t."
  ],
  "Stress": [
    "I have so much to do and not enough time.",
    "I feel like I’m about to snap.",
    "Everything feels like too much lately.",
    "I can’t catch a break—there’s always something new.",
    "I’m constantly tired and overwhelmed.",
    "My head is pounding from all the pressure.",
    "Deadlines are suffocating me.",
    "I wish I could just pause everything for a while.",
    "Even the smallest tasks feel like huge mountains.",
    "I can’t seem to calm down anymore."
  ],
  "Suicidal": [
    "I don’t see a point in anything anymore.",
    "I wish I could just disappear forever.",
    "Everyone would be better off without me.",
    "The pain is too much—I can’t take it anymore.",
    "I’m tired of pretending everything’s okay.",
    "I’ve thought about ending it more times than I can count.",
    "It feels like nothing will ever get better.",
    "I don’t know how much longer I can hold on.",
    "I feel like I’m drowning in darkness.",
    "Living hurts more than anything else right now."
  ],
  "depression": [
    "I feel numb all the time.",
    "Nothing brings me joy anymore.",
    "Getting out of bed feels impossible some days.",
    "I feel so alone, even when I’m surrounded by people.",
    "I’m constantly exhausted, no matter how much I sleep.",
    "Life feels like a heavy weight I have to carry.",
    "I just don’t care about anything anymore.",
    "I feel worthless and hopeless.",
    "I don’t recognize myself anymore.",
    "It’s like I’m stuck in a dark hole with no way out."
  ],
  "mentalillness": [
    "People think I’m making it up, but it’s very real.",
    "I wish others understood what I go through every day.",
    "Mental illness is invisible, but it affects everything.",
    "Some days are good, but some days are unbearable.",
    "Just because I look okay doesn’t mean I am.",
    "I feel ashamed for something I can’t control.",
    "I hate the stigma around mental illness.",
    "I wish I could explain how much I struggle inside.",
    "Therapy helps, but it’s still a battle.",
    "It’s not weakness—it’s an illness like any other."
  ],
  "bipolar": [
    "One week I feel unstoppable, the next I can’t get out of bed.",
    "I hate these extreme mood swings.",
    "My energy levels fluctuate so much it’s exhausting.",
    "People don’t understand how hard it is to live like this.",
    "Sometimes I feel like I can conquer the world, other times I want to disappear.",
    "I miss stability more than anything.",
    "Mania feels amazing—until the crash comes.",
    "Depression hits harder after the highs.",
    "I feel like two different people.",
    "Trying to find balance is a constant fight."
  ],
  "schizophrenia": [
    "I hear voices that no one else hears.",
    "Sometimes I can’t tell what’s real and what’s not.",
    "My mind plays tricks on me.",
    "I feel disconnected from reality.",
    "People think I’m dangerous, but I’m just scared.",
    "I wish the hallucinations would stop.",
    "It’s hard to trust anyone—including myself.",
    "Medication helps, but the side effects are rough.",
    "I live in a different world than everyone else.",
    "It’s exhausting to explain my reality to others."
  ],
  "Normal": [
    "I feel pretty balanced today.",
    "Life is going well at the moment.",
    "I have stress sometimes, but I’m managing fine.",
    "I enjoy spending time with my friends and family.",
    "I feel motivated and focused lately.",
    "I’ve been sleeping well and eating right.",
    "I feel content and at peace.",
    "I’m excited about the future.",
    "I’ve been feeling more confident lately.",
    "I feel like I’m in control of my life."
  ]
}

In [15]:
score = 0
max = 0

for status in msgs:
  cnt = 0
  print(f"Evaluate Model : Status => {status} :")
  for msg in msgs[status]:
    print(f"\tMessage : {msg}")
    res = predict_status(msg)
    print(f"\tPredicted : {res}")
    if(status.lower() == res.lower()):
      print("\t✅")
      cnt += 1
    else:
      print("\t❌")
  print(f"Status : {status} => Score : {cnt}/{len(msgs[status])}")
  score += cnt
  max += len(msgs[status])
  print()

print(f"Total Score : {score}/{max}")

Evaluate Model : Status => BPD :
	Message : I feel so intensely and then suddenly feel nothing at all.
	Predicted : BPD
	✅
	Message : One moment I trust you, the next I’m terrified you’ll leave me.
	Predicted : BPD
	✅
	Message : I hate myself for pushing people away, even when I need them most.
	Predicted : BPD
	✅
	Message : Why do my emotions change so fast? I can’t keep up.
	Predicted : BPD
	✅
	Message : Sometimes I just want someone to tell me they won’t leave.
	Predicted : Depression
	❌
	Message : I’m scared of abandonment even when nothing has happened.
	Predicted : BPD
	✅
	Message : My relationships always feel like a rollercoaster.
	Predicted : BPD
	✅
	Message : It’s like I don’t know who I really am.
	Predicted : BPD
	✅
	Message : Everything is either all good or all bad—there’s no in-between.
	Predicted : Bipolar
	❌
	Message : I feel broken and I don’t know how to fix it.
	Predicted : Mentalillness
	❌
Status : BPD => Score : 7/10

Evaluate Model : Status => Anxiety :
	Message 